In [2]:
"""
Source: https://stackoverflow.com/questions/77469097/how-can-i-upload-a-pdf-to-chatgpt-using-the-api
"""

from openai import OpenAI

from openai.types.beta.threads.message_create_params import (
    Attachment,
    AttachmentToolFileSearch,
)
import os

template_filename = "Industry at a Glance Template.pdf"
info_filename = "IT Consulting in the US.pdf"
prompt = "Extract section headers and subheader from the file provided."
prompt = """
Using the first document as a template, create a summary of the first document.
Split paragraphs into sentences preceded by the subject.
For example: "Revenue Growth: Revenue has grown at a CAGR of 2.8% to $692.9 billion over the past five years."
Return the result in JSON format.
"""

from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_APIKEY"))

pdf_assistant = client.beta.assistants.create(
    model="gpt-4o",
    description="An assistant to extract the contents of PDF files.",
    tools=[{"type": "file_search"}],
    name="PDF assistant",
)

# Create thread
thread = client.beta.threads.create()

info_file = client.files.create(file=open(info_filename, "rb"), purpose="assistants")
template_file = client.files.create(file=open(template_filename, "rb"), purpose="assistants")

# Create assistant
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    attachments=[
        Attachment(
            file_id=template_file.id, tools=[AttachmentToolFileSearch(type="file_search")]
        ),
        Attachment(
            file_id=info_file.id, tools=[AttachmentToolFileSearch(type="file_search")]
        )
    ],
    content=prompt,
    # response_format={"type": "json_object"},
)

# Run thread
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=pdf_assistant.id, timeout=1000
)

if run.status != "completed":
    raise Exception("Run failed:", run.status)

messages_cursor = client.beta.threads.messages.list(thread_id=thread.id)
messages = [message for message in messages_cursor]

TypeError: Messages.create() got an unexpected keyword argument 'response_format'